# This is the beginning of the DataHacks ScienceGang Project

## Part1: General Setup and Imports

In [48]:
# general imports
import os
import numpy as np
import pandas as pd
import seaborn as sns
import folium
from folium import plugins

# These lines set up graphing capabilities.
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
pd.set_option('display.max_rows', 15)

%matplotlib inline

In [2]:
# read .csv file into a DataFrame
loc = 'U.S._Chronic_Disease_Indicators__CDI_.csv'
Chronic_Diseases = pd.read_csv(loc)
Chronic_Diseases

,YearStart,YearEnd,LocationAbbr,LocationDesc,DataSource,Topic,Question,Response,DataValueUnit,DataValueType,...,LocationID,TopicID,QuestionID,DataValueTypeID,StratificationCategoryID1,StratificationID1,StratificationCategoryID2,StratificationID2,StratificationCategoryID3,StratificationID3
0,2015,2015,US,United States,APIS,Alcohol,Amount of alcohol excise tax by beverage type ...,NaN,$,US Dollars,...,59,ALC,ALC8_0_2,USD,OVERALL,OVR,NaN,NaN,NaN,NaN
1,2012,2012,DC,District of Columbia,APIS,Alcohol,Amount of alcohol excise tax by beverage type ...,NaN,$,US Dollars,...,11,ALC,ALC8_0_1,USD,OVERALL,OVR,NaN,NaN,NaN,NaN
2,2012,2012,GU,Guam,APIS,Alcohol,Amount of alcohol excise tax by beverage type ...,NaN,$,US Dollars,...,66,ALC,ALC8_0_1,USD,OVERALL,OVR,NaN,NaN,NaN,NaN
3,2012,2012,PR,Puerto Rico,APIS,Alcohol,Amount of alcohol excise tax by beverage type ...,NaN,$,US Dollars,...,72,ALC,ALC8_0_1,USD,OVERALL,OVR,NaN,NaN,NaN,NaN
4,2012,2012,US,United States,APIS,Alcohol,Amount of alcohol excise tax by beverage type ...,NaN,$,US Dollars,...,59,ALC,ALC8_0_1,USD,OVERALL,OVR,NaN,NaN,NaN,NaN
5,2012,2012,VI,Virgin Islands,APIS,Alcohol,Amount of alcohol excise tax by beverage type ...,NaN,$,US Dollars,...,78,ALC,ALC8_0_1,USD,OVERALL,OVR,NaN,NaN,NaN,NaN
6,2012,2012,GU,Guam,APIS,Alcohol,Amount of alcohol excise tax by beverage type ...,NaN,$,US Dollars,...,66,ALC,ALC8_0_2,USD,OVERALL,OVR,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519711,2014,2014,TN,Tennessee,State Inpatient Data,Asthma,Hospitalizations for asthma,NaN,"cases per 10,000",Crude Rate,...,47,AST,AST3_1,CRDRATE,GENDER,GENF,NaN,NaN,NaN,NaN
519712,2016,2016,DC,District of Columbia,BRFSS,Cardiovascular Disease,Pneumococcal vaccination among noninstitutiona...,NaN,%,Age-adjusted Prevalence,...,11,CVD,CVD10_1,AGEADJPREV,GENDER,GENF,NaN,NaN,NaN,NaN


In [3]:
# browse data structure and contents
print(Chronic_Diseases.shape)
print(Chronic_Diseases.describe())

(519718, 34)
           YearStart        YearEnd  DataValueAlt  LowConfidenceLimit  \
count  519718.000000  519718.000000  3.503350e+05       311062.000000   
mean     2013.141885    2013.162754  8.917742e+02           46.759401   
std         1.777622       1.762672  1.833073e+04           77.492628   
min      2001.000000    2001.000000  0.000000e+00            0.200000   
25%      2012.000000    2012.000000  1.845500e+01           12.700000   
50%      2013.000000    2013.000000  4.100000e+01           30.200000   
75%      2015.000000    2015.000000  7.030000e+01           55.400000   
max      2016.000000    2016.000000  2.600878e+06         1330.660000   

       HighConfidenceLimit     LocationID  
count        311062.000000  519718.000000  
mean             58.991287      30.993144  
std              88.668862      17.723341  
min               0.420000       1.000000  
25%              18.900000      17.000000  
50%              43.800000      30.000000  
75%              70.4

In [4]:
Chronic_Diseases.isna().sum()

YearStart                         0
YearEnd                           0
LocationAbbr                      0
LocationDesc                      0
DataSource                        0
Topic                             0
Question                          0
                              ...  
DataValueTypeID                   0
StratificationCategoryID1         0
StratificationID1                 0
StratificationCategoryID2    440529
StratificationID2            440529
StratificationCategoryID3    440530
StratificationID3            440530
Length: 34, dtype: int64

## Explore Contents in the Response Column

In [5]:
# number of missing values
Chronic_Diseases.get("Response").isna().sum()

440530

In [6]:
# All remaining non-missing responses are blank
Chronic_Diseases[Chronic_Diseases.get("Response").notna()].get("Response").value_counts()

     79188
Name: Response, dtype: int64

Since there is no meaningful data in the response column, it will be dropped.

In [7]:
Chronic_Diseases.drop(columns="Response", inplace=True)

## Explore DataSources

In [11]:
Chronic_Diseases.get("DataSource").isna().sum()

0

### The dataset contains information from *32* different sources

In [14]:
# This dataset consists of 32 various sources
Chronic_Diseases.get("DataSource").value_counts().shape

(32,)

## Explore Stratification Categories

### Stratification Category 1 contains `RACE`,`GENDER`,`OVERALL`

In [15]:
Chronic_Diseases.get("StratificationCategoryID1").value_counts()

RACE       320170
GENDER     121660
OVERALL     77888
Name: StratificationCategoryID1, dtype: int64

### Stratification Category 2 contains all `NULL` and `BLANK` values

In [24]:
print(Chronic_Diseases.get("StratificationCategoryID2").isna().sum())
print(Chronic_Diseases.get("StratificationCategoryID2").value_counts())

440529
     79189
Name: StratificationCategoryID2, dtype: int64


In [25]:
# Drop the Stratification ID2 column
Chronic_Diseases.drop(columns="StratificationCategoryID2",inplace=True)

### Stratification Category 3 also contains all `NULL` and `BLANK` values

In [27]:
print(Chronic_Diseases.get("StratificationCategoryID3").isna().sum())
Chronic_Diseases.get("StratificationCategoryID3").value_counts()

440530


     79188
Name: StratificationCategoryID3, dtype: int64

In [28]:
# Drop the Stratification ID3 column
Chronic_Diseases.drop(columns="StratificationCategoryID3",inplace=True)

## Explore Stratification IDs

### Stratification ID1 contains 11 subcategories relating to `RACE`, `GENDER`, `OVERALL`

In [30]:
print(Chronic_Diseases.get("StratificationID1").isna().sum())
print(Chronic_Diseases.get("StratificationID1").value_counts())

0


OVR     77888
HIS     64034
WHT     64034
BLK     64034
GENM    60830
GENF    60830
OTH     46441
MRC     46441
AIAN    17593
API     16851
ASN       742
Name: StratificationID1, dtype: int64

### Stratification ID2 contains all `NULL` and `BLANK`

In [31]:
print(Chronic_Diseases.get("StratificationID2").isna().sum())
print(Chronic_Diseases.get("StratificationID2").value_counts())

440529
     79189
Name: StratificationID2, dtype: int64


In [33]:
# Drop Stratification ID2 column
Chronic_Diseases.drop(columns="StratificationID2", inplace=True)

### Stratification ID3 contains all `NULL` and `BLANK`

In [32]:
print(Chronic_Diseases.get("StratificationID3").isna().sum())
print(Chronic_Diseases.get("StratificationID3").value_counts())

440530
     79188
Name: StratificationID3, dtype: int64


In [35]:
# Drop Stratification ID3 column
Chronic_Diseases.drop(columns="StratificationID3", inplace=True)

## Explore Stratification Numbers

In [50]:
print(Chronic_Diseases.get("StratificationCategory1").isna().sum())
print(Chronic_Diseases.get("StratificationCategory1").value_counts())
print(Chronic_Diseases.get("StratificationCategory2").isna().sum())
print(Chronic_Diseases.get("StratificationCategory2").value_counts())
print(Chronic_Diseases.get("StratificationCategory3").isna().sum())
print(Chronic_Diseases.get("StratificationCategory3").value_counts())

0
Race/Ethnicity    320170
Gender            121660
Overall            77888
Name: StratificationCategory1, dtype: int64
440530
     79188
Name: StratificationCategory2, dtype: int64
440530
     79188
Name: StratificationCategory3, dtype: int64


In [51]:
print(Chronic_Diseases.get("Stratification1").isna().sum())
print(Chronic_Diseases.get("Stratification1").value_counts())
print(Chronic_Diseases.get("Stratification2").isna().sum())
print(Chronic_Diseases.get("Stratification2").value_counts())
print(Chronic_Diseases.get("Stratification3").isna().sum())
print(Chronic_Diseases.get("Stratification3").value_counts())

0
Overall                             77888
Hispanic                            64034
Black, non-Hispanic                 64034
White, non-Hispanic                 64034
Male                                60830
Female                              60830
Multiracial, non-Hispanic           46441
Other, non-Hispanic                 46441
American Indian or Alaska Native    17593
Asian or Pacific Islander           16851
Asian, non-Hispanic                   742
Name: Stratification1, dtype: int64
440530
     79188
Name: Stratification2, dtype: int64
440530
     79188
Name: Stratification3, dtype: int64


In [52]:
Chronic_Diseases.drop(columns=['Stratification2','Stratification3','StratificationCategory2','StratificationCategory3'],inplace=True)

## Next, let's see if there are redundant columns

In [53]:
Chronic_Diseases.columns

Index(['YearStart', 'YearEnd', 'LocationAbbr', 'LocationDesc', 'DataSource',
       'Topic', 'Question', 'DataValueUnit', 'DataValueType', 'DataValue',
       'DataValueAlt', 'DataValueFootnoteSymbol', 'DatavalueFootnote',
       'LowConfidenceLimit', 'HighConfidenceLimit', 'StratificationCategory1',
       'Stratification1', 'GeoLocation', 'ResponseID', 'LocationID', 'TopicID',
       'QuestionID', 'DataValueTypeID', 'StratificationCategoryID1',
       'StratificationID1'],
      dtype='object')

In [54]:
stratification_category = pd.DataFrame().assign(c1=Chronic_Diseases.get("StratificationCategory1"), c2=Chronic_Diseases.get("StratificationCategoryID1"))
stratification_category

,c1,c2
0,Overall,OVERALL
1,Overall,OVERALL
2,Overall,OVERALL
3,Overall,OVERALL
4,Overall,OVERALL
5,Overall,OVERALL
6,Overall,OVERALL
...,...,...
519711,Gender,GENDER
519712,Gender,GENDER
